In [1]:
import os
import openai
api_key = os.getenv("openai_api_key")

In [2]:
import psycopg2

In [11]:
import psycopg2
import openai
import pandas as pd
import json

db_params = {
    "dbname": "aihistory",
    "user": "postgres",
    "password": "4555",
    "host": "localhost",
    "port": "5432"
}
openai.api_key = api_key
try:
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()
    print("Connected to the database successfully!")
except psycopg2.OperationalError as e:
    print("Operational error:", e)

Connected to the database successfully!


In [12]:
csv_file_path = r'C:\Users\Dell User\Desktop\New-RAG\history_of_ai.csv'
df = pd.read_csv(csv_file_path)
try:
    for _, row in df.iterrows():
        cursor.execute(
            "INSERT INTO mydata (year, event, description, impact) VALUES (%s, %s, %s, %s)",
            (row['Year'], row['Event'], row['Description'], row['Impact'])
        )
    conn.commit()
    print(f"{len(df)} rows inserted into the table successfully.")
except Exception as e:
    print("Error inserting data:", e)

6 rows inserted into the table successfully.


In [13]:
import psycopg2
import openai
import json

db_params = {
    "dbname": "aihistory",
    "user": "postgres",
    "password": "4555",
    "host": "localhost",
    "port": "5432"
}

openai.api_key = api_key
def generate_embeddings(text):
    try:
        response = openai.Embedding.create(
            input=text,
            model="text-embedding-ada-002"
        )
        return response['data'][0]['embedding']
    except openai.error.OpenAIError as e:
        print(f"OpenAI API error: {e}")
        return None

try:
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()
    print("Connected to the database successfully!")
    cursor.execute("SELECT year, description FROM mydata WHERE embeddings IS NULL;")
    rows = cursor.fetchall()

    for i, row in enumerate(rows):
        year = row[0] 
        description = row[1]

        try:
            embedding = generate_embeddings(description)
            if embedding:
                cursor.execute(
                    "UPDATE mydata SET embeddings = %s WHERE year = %s;",
                    (json.dumps(embedding), year)
                )
                print(f"Updated embeddings for year: {year}")
            if i % 100 == 0:
                conn.commit()
                print(f"Committed after processing {i} records.")

        except Exception as update_error:
            print(f"Error updating record for year {year}: {update_error}")
            conn.rollback()
    conn.commit()
    print("All embeddings generated and stored successfully.")

except Exception as e:
    print(f"Error generating embeddings: {e}")

finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()
    print("Database connection closed.")

Connected to the database successfully!
Updated embeddings for year: 1956
Committed after processing 0 records.
Updated embeddings for year: 1966
Updated embeddings for year: 1980
Updated embeddings for year: 1997
Updated embeddings for year: 2011
Updated embeddings for year: 2023
All embeddings generated and stored successfully.
Database connection closed.


In [27]:
import numpy as np
import psycopg2
import openai
import json

def cosine_similarity(vec1, vec2):
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
def retrieve_similar_rows(query, top_k=2):
    query_embedding = openai.Embedding.create(
        input=query,
        model="text-embedding-ada-002"
    )['data'][0]['embedding']
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()
    cursor.execute("SELECT description, embeddings FROM mydata WHERE embeddings IS NOT NULL;")
    rows = cursor.fetchall()

    results = []
    for row in rows:
        description = row[0]
        embedding = row[1] 
        similarity = cosine_similarity(query_embedding, embedding)
        results.append((description, similarity))
    results = sorted(results, key=lambda x: x[1], reverse=True)
    conn.close()
    return results[:top_k]

query = "What are the key milestones in AI history?"
top_results = retrieve_similar_rows(query)
for i, (description, score) in enumerate(top_results, 1):
    print(f"Top {i} (Similarity: {score:.4f}):\n{description}\n")

Top 1 (Similarity: 0.8459):
The term 'Artificial Intelligence' was coined at the Dartmouth Conference, marking the birth of AI as a field of study.

Top 2 (Similarity: 0.8272):
Advancements in generative models like GPT and Stable Diffusion transformed AI's ability to create text, images, and more.



In [28]:
def ask_question(question):
    top_result = retrieve_similar_rows(question)
    if not top_result:
        return "I'm sorry, I couldn't find relevant information in the database."
    top_description, _ = top_result[0]
    context = f"- {top_description}"
    chain = LLMChain(llm=llm, prompt=retrieval_prompt)
    response = chain.run({"question": question, "context": context})
    return response.strip()


user_question = "What are the most significant milestones in AI history?"
response = ask_question(user_question)
print(response)

Some of the most significant milestones in AI history include the coining of the term 'Artificial Intelligence' at the Dartmouth Conference in 1956, the development of the first AI programs in the 1950s and 1960s, the creation of expert systems in the 1970s and 1980s, and the resurgence of AI in the 21st century with advancements in machine learning and deep learning technologies.
